In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as stats
from scipy.stats import binom
import scipy
import itertools
from collections import Counter
from collections import defaultdict
import functools
import Bio
from Bio import motifs
import multiprocessing as mp
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn import ensemble
import math
import bokeh
from bokeh.charts import Chord
from bokeh.io import show, output_file
%matplotlib inline

C:\Users\Wolf\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Retreive previously calculated tres for dataset
sm_csv = 'c:/users/wolf/desktop/SynPro/tre_analysis_selectedseqs.csv'
#sm_csv = 'Z:/PROJECTS/Synthetic Promoters/Library#1/Hits Testing/p7_tredf.csv'
sm_df = pd.read_csv(sm_csv)
sm_df['seq_len'] = sm_df['Sequence'].str.len()
sm_df.head()

,Name,Sequence,tre,position,logprob,seq_len
0,S1-17,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGAGGAA...,E2F1,-62,10.755157,265
1,S1-17,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGAGGAA...,SP1,189,14.480309,265
2,S1-17,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGAGGAA...,FOS::JUN,56,11.726842,265
3,S1-17,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGAGGAA...,FOS::JUN,-89,11.726842,265
4,S1-17,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGAGGAA...,REL,-182,13.255016,265


In [3]:
#libaddress = 'Z:/PROJECTS/Synthetic Promoters/Library#1/Hits Testing/Jaspar_lib.txt'
libaddress = 'C:/Users/Wolf/Documents/BioScripts/Jaspar_lib.txt'
fh = open(libaddress)

#Collect all TRE names, in order
with open(libaddress, 'r') as f:
    name_lst = []
    for ln in f:
        if ln.startswith('>'):
            name_lst.append(ln[10:-1])

#get frequency matrix for each TRE
pssms = [m.counts.normalize(pseudocounts=0.25).log_odds() for m in motifs.parse(fh, "jaspar")]

In [4]:
#Calc length of each TRE, put into df
tre_lens = [len(pwm[0]) for pwm in pssms]
tre_len_dict = list(zip(name_lst, tre_lens))
trelen_df = pd.DataFrame(tre_len_dict).reset_index()
trelen_df.columns = ['tre', 'tre_name', 'tre_len']
del trelen_df['tre']
trelen_df.head()

,tre_name,tre_len
0,RUNX1,11
1,TFAP2A,11
2,Arnt,6
3,Ahr::Arnt,6
4,Ar,17


In [5]:
jsel_df = pd.merge(sm_df, trelen_df, right_on='tre_name', left_on='tre', how='inner')
jsel_df.head()

,Name,Sequence,tre,position,logprob,seq_len,tre_name,tre_len
0,S1-17,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGAGGAA...,E2F1,-62,10.755157,265,E2F1,12
1,S1-37,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGGGGCG...,E2F1,156,10.755157,232,E2F1,12
2,S1-1,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGATCTCCGC...,E2F1,119,10.755157,256,E2F1,12
3,S1-3,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGAGGGGACT...,E2F1,68,10.755157,205,E2F1,12
4,S1-42,CTGCTTAGGGTTAGGCGTTTTGCGCTGCTTCGCGATCGATTTCCAA...,E2F1,-122,10.755157,283,E2F1,12


In [6]:
grp_df = jsel_df[['Name', 'tre']].groupby(['Name', 'tre']).size().reset_index()
grp_df.columns = ['Name', 'tre', 'count']
pivdf = grp_df.pivot(index='Name', columns= 'tre').reset_index()
pivdf.columns = pivdf.columns.droplevel(0)
pivdf.rename(columns={'':'Name'}, inplace=True)
pivdf.fillna(value=0, inplace=True)
pivdf.head()


tre,Name,ARNT::HIF1A,ATF7,BATF::JUN,Bcl6,CTCF,CUX1,CUX2,Creb5,E2F1,...,TBR1,TBX2,TCF7L2,TFAP2A(var.3),TFAP2B(var.3),TFAP2C(var.3),Tcf7,ZBTB7B,ZNF263,ZNF740
0,S1-1,0.0,4.0,1.0,2.0,0.0,0.0,0.0,4.0,1.0,...,1.0,1.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0
1,S1-10,0.0,2.0,3.0,5.0,0.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,2.0,2.0,4.0,1.0,0.0,0.0,1.0
2,S1-14,0.0,6.0,2.0,2.0,0.0,4.0,4.0,6.0,2.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,S1-15,1.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,2.0,2.0,4.0,1.0,0.0,1.0,1.0
4,S1-16,0.0,6.0,1.0,6.0,1.0,0.0,0.0,6.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
geomean_address = 'c:/users/wolf/desktop/synpro/Tcell_geomean.csv'
geomean_df = pd.read_csv(geomean_address)
geomean_df['delta'] = geomean_df.stim - geomean_df.baseline
geomean_df.head(10)

,sample,baseline,stim,delta
0,NaN,112.0,201.0,89.0
1,CD19CAR alone,79.3,89.7,10.4
2,No promoter,112.0,114.0,2.0
3,IL2mp only,137.0,144.0,7.0
4,6xNFAT,125.0,189.0,64.0
5,7xNFkB,195.0,802.0,607.0
6,S2-n1,156.0,274.0,118.0
7,S4-n1,334.0,868.0,534.0
8,S6-n1,136.0,154.0,18.0
9,S1-17,219.0,682.0,463.0


In [8]:
intmer1 = pd.merge(geomean_df[['sample', 'delta']], jsel_df[['Name', 'seq_len']], left_on='sample', right_on='Name', how='inner')
del intmer1['sample']
intmer1.drop_duplicates(inplace=True)

anadf = pd.merge(intmer1, pivdf, on='Name', how='inner')
del anadf['Name']
anadf.rename(columns={'delta':'count'}, inplace=True)
anadf.head()

,count,seq_len,ARNT::HIF1A,ATF7,BATF::JUN,Bcl6,CTCF,CUX1,CUX2,Creb5,...,TBR1,TBX2,TCF7L2,TFAP2A(var.3),TFAP2B(var.3),TFAP2C(var.3),Tcf7,ZBTB7B,ZNF263,ZNF740
0,118.0,235,0.0,2.0,1.0,4.0,0.0,0.0,0.0,2.0,...,1.0,1.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0
1,534.0,295,0.0,2.0,0.0,2.0,0.0,3.0,3.0,2.0,...,1.0,1.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0
2,18.0,308,0.0,2.0,0.0,4.0,0.0,2.0,2.0,2.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,463.0,265,1.0,2.0,2.0,3.0,0.0,0.0,0.0,2.0,...,1.0,1.0,2.0,2.0,2.0,4.0,2.0,0.0,0.0,1.0
4,90.0,232,1.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,...,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0


In [9]:
trean_df = trelen_df

#retreive denominator for singlet global probability
tre_denoms = []
for t in trean_df['tre_len'].values:
    t_dnom = 0
    for l in anadf['seq_len'].values:
        seq_denom = (l-t)
        t_dnom += seq_denom
    tre_denoms.append(t_dnom)
trean_df['denom'] = tre_denoms

#retreive global singlet counts (numerator)
tre_cnt = pivdf.sum().reset_index()
tre_cnt.columns = ['tre_name', 'tre_count']

#calc singlet global probability
singlet_df = pd.merge(trean_df, tre_cnt, on='tre_name', how='inner')
singlet_df['singlet_prob'] = singlet_df.tre_count/ singlet_df.denom

print(singlet_df.shape)
singlet_df.head()

(70, 5)


,tre_name,tre_len,denom,tre_count,singlet_prob
0,E2F1,12,7864,31,0.00394201
1,Klf4,10,7926,25,0.00315418
2,PPARG,20,7616,6,0.000787815
3,SP1,11,7895,53,0.00671311
4,FOS::JUN,7,8019,43,0.00536226


In [10]:
possible_dublets = list((itertools.combinations(pivdf.columns[1:], 2)))
print(possible_dublets)

[('ARNT::HIF1A', 'ATF7'), ('ARNT::HIF1A', 'BATF::JUN'), ('ARNT::HIF1A', 'Bcl6'), ('ARNT::HIF1A', 'CTCF'), ('ARNT::HIF1A', 'CUX1'), ('ARNT::HIF1A', 'CUX2'), ('ARNT::HIF1A', 'Creb5'), ('ARNT::HIF1A', 'E2F1'), ('ARNT::HIF1A', 'EGR1'), ('ARNT::HIF1A', 'EGR3'), ('ARNT::HIF1A', 'EGR4'), ('ARNT::HIF1A', 'EOMES'), ('ARNT::HIF1A', 'ESR2'), ('ARNT::HIF1A', 'FOS'), ('ARNT::HIF1A', 'FOS::JUN'), ('ARNT::HIF1A', 'FOSL1'), ('ARNT::HIF1A', 'FOSL2'), ('ARNT::HIF1A', 'HNF4G'), ('ARNT::HIF1A', 'HSF4'), ('ARNT::HIF1A', 'IRF7'), ('ARNT::HIF1A', 'JDP2'), ('ARNT::HIF1A', 'JDP2(var.2)'), ('ARNT::HIF1A', 'JUN'), ('ARNT::HIF1A', 'JUN(var.2)'), ('ARNT::HIF1A', 'JUNB'), ('ARNT::HIF1A', 'JUND'), ('ARNT::HIF1A', 'JUND(var.2)'), ('ARNT::HIF1A', 'KLF14'), ('ARNT::HIF1A', 'KLF16'), ('ARNT::HIF1A', 'KLF5'), ('ARNT::HIF1A', 'Klf12'), ('ARNT::HIF1A', 'Klf4'), ('ARNT::HIF1A', 'LEF1'), ('ARNT::HIF1A', 'LIN54'), ('ARNT::HIF1A', 'MEIS3'), ('ARNT::HIF1A', 'NFATC2'), ('ARNT::HIF1A', 'NFE2'), ('ARNT::HIF1A', 'NR2C2'), ('ARNT::H

In [11]:
#create dataframe with presence/absence of dublets for each sequence
dublet_df = pd.DataFrame()
for tup in possible_dublets:
    dublet_df[tup] = np.where((pivdf[tup[0]].values > 0) & (pivdf[tup[1]].values > 0), 1, 0)
dublet_df.head()

,"(ARNT::HIF1A, ATF7)","(ARNT::HIF1A, BATF::JUN)","(ARNT::HIF1A, Bcl6)","(ARNT::HIF1A, CTCF)","(ARNT::HIF1A, CUX1)","(ARNT::HIF1A, CUX2)","(ARNT::HIF1A, Creb5)","(ARNT::HIF1A, E2F1)","(ARNT::HIF1A, EGR1)","(ARNT::HIF1A, EGR3)",...,"(TFAP2C(var.3), Tcf7)","(TFAP2C(var.3), ZBTB7B)","(TFAP2C(var.3), ZNF263)","(TFAP2C(var.3), ZNF740)","(Tcf7, ZBTB7B)","(Tcf7, ZNF263)","(Tcf7, ZNF740)","(ZBTB7B, ZNF263)","(ZBTB7B, ZNF740)","(ZNF263, ZNF740)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,1,0,1,1,0,0,1,0,...,1,0,1,1,0,1,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [12]:
#store len of singlet TREs in a callable dict
len_dict = dict(zip(trelen_df['tre_name'], trelen_df['tre_len']))
singlet_dict = dict(zip(singlet_df['tre_name'], singlet_df['singlet_prob']))

In [13]:
#add weight and length variables, calc singlet probability FOR A SEQUENCE
dublet_df['weight'] = anadf['count']
dublet_df['length'] = anadf['seq_len']
for k, v in singlet_dict.items():
    dublet_df[k] = (1-((1-v)**(dublet_df['length'] - len_dict[k])))
dublet_df.head()

,"(ARNT::HIF1A, ATF7)","(ARNT::HIF1A, BATF::JUN)","(ARNT::HIF1A, Bcl6)","(ARNT::HIF1A, CTCF)","(ARNT::HIF1A, CUX1)","(ARNT::HIF1A, CUX2)","(ARNT::HIF1A, Creb5)","(ARNT::HIF1A, E2F1)","(ARNT::HIF1A, EGR1)","(ARNT::HIF1A, EGR3)",...,PAX3,SMAD3,EOMES,TBR1,TFAP2B(var.3),TFAP2C(var.3),ATF7,Creb5,NFE2,TFAP2A(var.3)
0,0,0,0,0,0,0,0,0,0,0,...,0.400439,0.838650,0.608296,0.608882,0.268002,0.464414,0.807767,0.808105,0.705756,0.268002
1,0,0,0,0,0,0,0,0,0,0,...,0.476895,0.900801,0.695949,0.695498,0.327192,0.547582,0.877146,0.876926,0.787970,0.327192
2,0,0,0,0,0,0,0,0,0,0,...,0.492130,0.910724,0.712187,0.711574,0.339372,0.563825,0.888503,0.888218,0.802502,0.339372
3,0,1,1,0,1,1,0,0,1,0,...,0.439970,0.873486,0.654894,0.654897,0.298221,0.507751,0.846323,0.846321,0.750223,0.298221
4,0,0,0,0,0,0,0,0,0,0,...,0.396336,0.834678,0.603304,0.603956,0.264909,0.459875,0.803416,0.803795,0.700896,0.264909


In [14]:
#query tuple headers with headers of their elements, subtract multiplied singlet probabilities from dublet binaries
tuple_columns = [col for col in dublet_df.columns if isinstance(col, tuple)]
probs_success_samples = pd.concat([dublet_df['weight'] * dublet_df[col[0]] * dublet_df[col[1]] for col in tuple_columns], axis=1)
#probs_success_samples = pd.concat([dublet_df[col[0]] * dublet_df[col[1]] for col in tuple_columns], axis=1)
probs_success_samples.columns = tuple_columns
#probs_success_samples = probs_success_samples.loc[:, (probs_success_samples != 0).all()]

var_probs_samples = pd.concat([dublet_df['weight'] * (dublet_df[col[0]] * dublet_df[col[1]]) * (1 - (dublet_df[col[0]] * dublet_df[col[1]])) for col in tuple_columns], axis=1)
var_probs_samples.columns = tuple_columns
#var_probs_samples = var_probs_samples.loc[:, (var_probs_samples != 0).all()]
    
observed_success_samples = pd.concat([dublet_df['weight'] * dublet_df[col] for col in tuple_columns], axis=1)
observed_success_samples.columns = tuple_columns
#observed_success_samples.apply(pd.to_numeric, errors='coerce')
#observed_success_samples_new = pd.DataFrame()
#for col in observed_success_samples.columns:
#    if observed_success_samples[col].sum() != 0:
#        observed_success_samples_new[col] = observed_success_samples[col]
#observed_success_samples_new = observed_success_samples.loc[:, (observed_success_samples != 0).all()] #for some reason this is bugged

In [15]:
#given two datasets (expected vs observed), calculate normal CDF of expected and return p-value for observed
def norm_hypothesis_test(mean_probability_df, var_probability_df, success_df):
    u = mean_probability_df.sum()
    o = var_probability_df.sum()
    return scipy.stats.norm(u, o).cdf(success_df.sum())
    
#Calculate and sort p-values for a tuple of TREs
p_values = pd.DataFrame(norm_hypothesis_test(probs_success_samples, var_probs_samples, observed_success_samples))
p_values.columns = ['p_value']
p_values['tuples'] = observed_success_samples.columns
p_values.sort_values('p_value', ascending=True, inplace=True)

In [16]:
forchord = p_values.loc[p_values['p_value'] < 0.05]
forchord.shape

(125, 2)

In [17]:
forchord = p_values.loc[p_values['p_value'] < 0.05]
forchord['nameA'] = [x[0] for x in forchord['tuples'].values]
forchord['nameB'] = [x[1] for x in forchord['tuples'].values]
forchord['invp'] = [int((0.05-x)*100) for x in forchord['p_value'].values]
duplicatelst = ['REL', 'CUX2', 'TFAP2B(var.3)', 'TFAP2C(var.3)']
for n in duplicatelst:
    forchord = forchord.loc[(forchord['nameA'] != n) & (forchord['nameB'] != n)]
    print(forchord.shape)
forchord.head()

(117, 5)
(117, 5)
(117, 5)
(106, 5)


C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,p_value,tuples,nameA,nameB,invp
1274,0.000133,"(JDP2, Stat4)",JDP2,Stat4,4
1024,0.000134,"(FOSL1, Stat4)",FOSL1,Stat4,4
917,0.000134,"(FOS, Stat4)",FOS,Stat4,4
1076,0.000134,"(FOSL2, Stat4)",FOSL2,Stat4,4
1499,0.000134,"(JUND, Stat4)",JUND,Stat4,4


In [19]:
from bokeh.palettes import inferno
chord_novalinf = Chord(forchord, source='nameA', target='nameB', palette= inferno(2))
output_file('chord_novalinf.html', mode="inline")
show(chord_novalinf)